In [13]:
import pandas as pd
import numpy as np
import imageio
import requests 

from bs4 import BeautifulSoup
from time import sleep
from requests import get
from random import randint

from urllib import error

In [14]:
# target url in craigslist we want to scrape the clothing and accesories posts from
tokyo_url = "https://tokyo.craigslist.org/d/clothing-accessories/search/cla?lang=en&cc=us"

In [15]:
# getting the response through BeatifulSoup
response = get(tokyo_url)
html_soup = BeautifulSoup(response.text)

In [16]:
# we look for the total amount of posts
number_of = html_soup.find("div", {"class:", "search-legend"})
maximum = int(number_of.find("span", {"class:", "totalcount"}).text)
print(maximum)

409


In [17]:
# setting the base url for the images we are about to scrape
image_base_url = 'https://images.craigslist.org/{}_300x300.jpg'

pages = np.arange(0, maximum+1, 120)
print(pages)

# emtpy list to store the scraped data
city_data = []

# looping through the pages and making sure we only check the posts
# containing images, we also force the posts to be shown in English
for page in pages:
    response = get("https://tokyo.craigslist.org/d/clothing-accessories/search/cla?"
                  + "s="
                  + str(page)
                  + "&hasPic=1"
                  + "&lang=en&cc=us")
    
    # we set a modest sleep time between pages to make sure craigslist doesn´t
    # block us from accesing it
    sleep(randint(2, 4))
    
    # we extract the html content of the page
    html = BeautifulSoup(response.content)

    posts = html.find_all('li', class_='result-row')
    
    # looping through the posts in the current page
    for post_id, post in enumerate(posts):
        print(page+post_id)
        # list storing the common information for one post
        common_post_information = []
        
        # post_id
        post_id = page+post_id
        common_post_information.append(post_id)

        # finding the title
        title = post.find("a", {"class":"result-title hdrlnk"})
        title_text = title.text
        # inserting the title
        common_post_information.append(title_text)
        print(title_text)

        # getting the price
        price = post.a.text.strip()
        # removing the yen sign
        price = int(price.replace("¥",""))
        common_post_information.append(price)
        print(price)
        
        # processing the neighborhood information
        # and assigning the value "unknown" when the
        # post doesn't contain the information
        neighborhood_to_insert = None
        neighborhood = post.find('span', {"class":"result-hood"})
        if neighborhood is not None:
            neighborhood = neighborhood.text.strip().replace("(","").replace(")","")
            neighborhood_to_insert = neighborhood
        if neighborhood_to_insert is None:
            neighborhood_to_insert = "unknown"
        print("neighborhood_to_insert:", neighborhood_to_insert)
        
        # inserting the neighborhood
        common_post_information.append(neighborhood_to_insert)

        # we then get the link for the post itself so we can
        # access its information
        link = title["href"]
        print(link)
        
        # getting the content of the post using the prior link
        response = requests.get(link)
        content = BeautifulSoup(response.content)
                
        # looking for a condition and inserting unknown if the
        # information was not provided
        condition_to_insert = None
        condition_groups = content.find_all('p', {'class': 'attrgroup'})
        
        if condition_groups is not None:
            for condition_group in condition_groups:
                conditions = condition_group.find_all('span')
                for condition in conditions:
                    condition_string = condition.text.strip().split(':')
                    if condition_string[0] == "condition":
                        print("condition:", condition_string[1])  
                        condition_to_insert = condition_string[1]
        if condition_to_insert is None:
            condition_to_insert = "unknown"
            print("condition:", condition_to_insert) 
        
        # inserting the condition
        common_post_information.append(condition_to_insert)
        
        # getting the list_of_images, as we are only checking the posts
        # containing images we know at least there will be one image in them
        images = post.find("a", { "class":"result-image gallery"})
        list_of_images = images['data-ids'].split(",")
        list_of_images
        len(list_of_images)
        print("common_post_information", common_post_information)
        # looping through the images in the post
        for image_id, image in enumerate(list_of_images):
            image_information = common_post_information.copy()
            image = image.replace("1:", "")
            image_to_save_url = image_base_url.format(image)
            print(image)
            image_base_url = image_base_url
            print(image_to_save_url)
            # try to read the image url
            try:
                image_to_save = imageio.imread(image_to_save_url)
            # if there is an HTTPError just skip the image and jump to the next one
            except error.HTTPError as e:
                print("HTTP Error, erasing the entry")
                continue
                
            name_of_image = "images/" + str(post_id) + "_" + image + "_" + str(image_id) + ".jpg"
            imageio.imwrite(name_of_image, image_to_save)

            # append image_id to the list
            image_information.append(image_id)
            # append image uri to the list
            image_information.append(name_of_image)
            print("image_information", image_information)
            city_data.append(image_information)

[  0 120 240 360]
0
Stitch-On Bag for sale! Brand New, Office worker, students OK
8000
neighborhood_to_insert: Ikebukuro
https://tokyo.craigslist.org/clo/d/stitch-on-bag-for-sale-brand-new-office/7017145428.html?lang=en&cc=us
condition:  like new
common_post_information [0, 'Stitch-On Bag for sale! Brand New, Office worker, students OK', 8000, 'Ikebukuro', ' like new']
00606_e20sHrQkKN1
https://images.craigslist.org/00606_e20sHrQkKN1_300x300.jpg
image_information [0, 'Stitch-On Bag for sale! Brand New, Office worker, students OK', 8000, 'Ikebukuro', ' like new', 0, 'images/0_00606_e20sHrQkKN1_0.jpg']
00C0C_jT99a4x5PXI
https://images.craigslist.org/00C0C_jT99a4x5PXI_300x300.jpg
image_information [0, 'Stitch-On Bag for sale! Brand New, Office worker, students OK', 8000, 'Ikebukuro', ' like new', 1, 'images/0_00C0C_jT99a4x5PXI_1.jpg']
00K0K_gsFZSt3KiSJ
https://images.craigslist.org/00K0K_gsFZSt3KiSJ_300x300.jpg
image_information [0, 'Stitch-On Bag for sale! Brand New, Office worker, stude

condition: unknown
common_post_information [4, 'J-Tech Tactical Backpack', 10000, 'unknown', 'unknown']
00303_awzghRGbR4R
https://images.craigslist.org/00303_awzghRGbR4R_300x300.jpg
image_information [4, 'J-Tech Tactical Backpack', 10000, 'unknown', 'unknown', 0, 'images/4_00303_awzghRGbR4R_0.jpg']
00K0K_lmtpzJk9YRx
https://images.craigslist.org/00K0K_lmtpzJk9YRx_300x300.jpg
image_information [4, 'J-Tech Tactical Backpack', 10000, 'unknown', 'unknown', 1, 'images/4_00K0K_lmtpzJk9YRx_1.jpg']
00R0R_bdvyv2uqfqd
https://images.craigslist.org/00R0R_bdvyv2uqfqd_300x300.jpg
image_information [4, 'J-Tech Tactical Backpack', 10000, 'unknown', 'unknown', 2, 'images/4_00R0R_bdvyv2uqfqd_2.jpg']
00z0z_f2ESXjO4KUn
https://images.craigslist.org/00z0z_f2ESXjO4KUn_300x300.jpg
image_information [4, 'J-Tech Tactical Backpack', 10000, 'unknown', 'unknown', 3, 'images/4_00z0z_f2ESXjO4KUn_3.jpg']
00k0k_eS8u9f7FHeL
https://images.craigslist.org/00k0k_eS8u9f7FHeL_300x300.jpg
image_information [4, 'J-Tech Tact

condition:  excellent
common_post_information [10, 'One of a kind designer leather jacket.', 35000, 'Sangenjaya', ' excellent']
00q0q_8whzdnt7P0W
https://images.craigslist.org/00q0q_8whzdnt7P0W_300x300.jpg
image_information [10, 'One of a kind designer leather jacket.', 35000, 'Sangenjaya', ' excellent', 0, 'images/10_00q0q_8whzdnt7P0W_0.jpg']
00m0m_a3Ruu8l6xqe
https://images.craigslist.org/00m0m_a3Ruu8l6xqe_300x300.jpg
image_information [10, 'One of a kind designer leather jacket.', 35000, 'Sangenjaya', ' excellent', 1, 'images/10_00m0m_a3Ruu8l6xqe_1.jpg']
00M0M_i1hCAgGWHDC
https://images.craigslist.org/00M0M_i1hCAgGWHDC_300x300.jpg
image_information [10, 'One of a kind designer leather jacket.', 35000, 'Sangenjaya', ' excellent', 2, 'images/10_00M0M_i1hCAgGWHDC_2.jpg']
00202_1b65iwhm4pU
https://images.craigslist.org/00202_1b65iwhm4pU_300x300.jpg
image_information [10, 'One of a kind designer leather jacket.', 35000, 'Sangenjaya', ' excellent', 3, 'images/10_00202_1b65iwhm4pU_3.jpg']


image_information [16, 'Monclear winter jacket', 70000, 'ebisu', ' excellent', 3, 'images/16_00w0w_dKQAc8dpXkl_3.jpg']
00z0z_8fBxe3JdUSZ
https://images.craigslist.org/00z0z_8fBxe3JdUSZ_300x300.jpg
image_information [16, 'Monclear winter jacket', 70000, 'ebisu', ' excellent', 4, 'images/16_00z0z_8fBxe3JdUSZ_4.jpg']
00L0L_iKjsf2v0NqM
https://images.craigslist.org/00L0L_iKjsf2v0NqM_300x300.jpg
image_information [16, 'Monclear winter jacket', 70000, 'ebisu', ' excellent', 5, 'images/16_00L0L_iKjsf2v0NqM_5.jpg']
00Q0Q_fcWvLQMuV1a
https://images.craigslist.org/00Q0Q_fcWvLQMuV1a_300x300.jpg
image_information [16, 'Monclear winter jacket', 70000, 'ebisu', ' excellent', 6, 'images/16_00Q0Q_fcWvLQMuV1a_6.jpg']
00S0S_bhHpQBWYbNu
https://images.craigslist.org/00S0S_bhHpQBWYbNu_300x300.jpg
image_information [16, 'Monclear winter jacket', 70000, 'ebisu', ' excellent', 7, 'images/16_00S0S_bhHpQBWYbNu_7.jpg']
00o0o_dbmB6Rq1en0
https://images.craigslist.org/00o0o_dbmB6Rq1en0_300x300.jpg
image_informati

image_information [23, "Nike Winter Men's Down Jacket", 4000, 'JR Matsudo station', ' like new', 2, 'images/23_01111_9vtKRrNJrD2_2.jpg']
00o0o_iurLOv27haG
https://images.craigslist.org/00o0o_iurLOv27haG_300x300.jpg
image_information [23, "Nike Winter Men's Down Jacket", 4000, 'JR Matsudo station', ' like new', 3, 'images/23_00o0o_iurLOv27haG_3.jpg']
00v0v_boILZb3Q29X
https://images.craigslist.org/00v0v_boILZb3Q29X_300x300.jpg
image_information [23, "Nike Winter Men's Down Jacket", 4000, 'JR Matsudo station', ' like new', 4, 'images/23_00v0v_boILZb3Q29X_4.jpg']
24
Dark Brown Leather/Suede Handbag
2000
neighborhood_to_insert: Ichigaya
https://tokyo.craigslist.org/clo/d/dark-brown-leather-suede-handbag/7016573187.html?lang=en&cc=us
condition:  excellent
common_post_information [24, 'Dark Brown Leather/Suede Handbag', 2000, 'Ichigaya', ' excellent']
00e0e_5b1wMlKKTEn
https://images.craigslist.org/00e0e_5b1wMlKKTEn_300x300.jpg
image_information [24, 'Dark Brown Leather/Suede Handbag', 2000,

...

condition: unknown
common_post_information [366, 'Women’s suit jacket and skirt jp size 11', 2000, 'Nishikoyama Station', 'unknown']
00U0U_kadATt5FnnK
https://images.craigslist.org/00U0U_kadATt5FnnK_300x300.jpg
image_information [366, 'Women’s suit jacket and skirt jp size 11', 2000, 'Nishikoyama Station', 'unknown', 0, 'images/366_00U0U_kadATt5FnnK_0.jpg']
00505_fAcOxWCFcC
https://images.craigslist.org/00505_fAcOxWCFcC_300x300.jpg
image_information [366, 'Women’s suit jacket and skirt jp size 11', 2000, 'Nishikoyama Station', 'unknown', 1, 'images/366_00505_fAcOxWCFcC_1.jpg']
00F0F_aUOMJZ6wClL
https://images.craigslist.org/00F0F_aUOMJZ6wClL_300x300.jpg
image_information [366, 'Women’s suit jacket and skirt jp size 11', 2000, 'Nishikoyama Station', 'unknown', 2, 'images/366_00F0F_aUOMJZ6wClL_2.jpg']
367
Dakine helipack 15 liters backpack
2000
neighborhood_to_insert: unknown
https://tokyo.craigslist.org/clo/d/dakine-helipack-15-liters-backpack/6992312233.html?lang=en&cc=us
condition: un

image_information [376, 'Alexander McQueen Men’s Coat', 40000, 'Kawasaki', ' excellent', 2, 'images/376_01717_jfZxtKN6ECb_2.jpg']
00x0x_14Hqki6mcjf
https://images.craigslist.org/00x0x_14Hqki6mcjf_300x300.jpg
image_information [376, 'Alexander McQueen Men’s Coat', 40000, 'Kawasaki', ' excellent', 3, 'images/376_00x0x_14Hqki6mcjf_3.jpg']
377
Hermes Mens Coat Authentic
50000
neighborhood_to_insert: Kawasaki
https://tokyo.craigslist.org/clo/d/hermes-mens-coat-authentic/6990885638.html?lang=en&cc=us
condition: unknown
common_post_information [377, 'Hermes Mens Coat Authentic', 50000, 'Kawasaki', 'unknown']
00p0p_bPcGWaMEFAW
https://images.craigslist.org/00p0p_bPcGWaMEFAW_300x300.jpg
image_information [377, 'Hermes Mens Coat Authentic', 50000, 'Kawasaki', 'unknown', 0, 'images/377_00p0p_bPcGWaMEFAW_0.jpg']
00B0B_is60FfA5IPK
https://images.craigslist.org/00B0B_is60FfA5IPK_300x300.jpg
image_information [377, 'Hermes Mens Coat Authentic', 50000, 'Kawasaki', 'unknown', 1, 'images/377_00B0B_is60F

condition: unknown
common_post_information [383, 'Brand new backpack from Haglofs, backup 15', 8500, 'Kyodo, Setagaya', 'unknown']
00Y0Y_eivERi4wzLg
https://images.craigslist.org/00Y0Y_eivERi4wzLg_300x300.jpg
image_information [383, 'Brand new backpack from Haglofs, backup 15', 8500, 'Kyodo, Setagaya', 'unknown', 0, 'images/383_00Y0Y_eivERi4wzLg_0.jpg']
00F0F_hrYS6zVW3wq
https://images.craigslist.org/00F0F_hrYS6zVW3wq_300x300.jpg
image_information [383, 'Brand new backpack from Haglofs, backup 15', 8500, 'Kyodo, Setagaya', 'unknown', 1, 'images/383_00F0F_hrYS6zVW3wq_1.jpg']
00R0R_5v7MrNrWh8J
https://images.craigslist.org/00R0R_5v7MrNrWh8J_300x300.jpg
image_information [383, 'Brand new backpack from Haglofs, backup 15', 8500, 'Kyodo, Setagaya', 'unknown', 2, 'images/383_00R0R_5v7MrNrWh8J_2.jpg']
384
Velvet Jacquard Boots from Asos
4000
neighborhood_to_insert: Bunky-ku
https://tokyo.craigslist.org/clo/d/velvet-jacquard-boots-from-asos/6988081240.html?lang=en&cc=us
condition:  like new
co

image_information [386, 'BRAND NEW!!!HUGE DISCOUNT!adidas originals women sneakers', 5700, 'Shinjuku', ' new', 4, 'images/386_00a0a_4GKuh2Tej76_4.jpg']
387
Zatchels saddle bag
8000
neighborhood_to_insert: Setagaya/Shibuya
https://tokyo.craigslist.org/clo/d/zatchels-saddle-bag/6988874403.html?lang=en&cc=us
condition:  like new
common_post_information [387, 'Zatchels saddle bag', 8000, 'Setagaya/Shibuya', ' like new']
00U0U_dhedz4fyQUr
https://images.craigslist.org/00U0U_dhedz4fyQUr_300x300.jpg
image_information [387, 'Zatchels saddle bag', 8000, 'Setagaya/Shibuya', ' like new', 0, 'images/387_00U0U_dhedz4fyQUr_0.jpg']
00Q0Q_lQh2isK4V0D
https://images.craigslist.org/00Q0Q_lQh2isK4V0D_300x300.jpg
image_information [387, 'Zatchels saddle bag', 8000, 'Setagaya/Shibuya', ' like new', 1, 'images/387_00Q0Q_lQh2isK4V0D_1.jpg']
00B0B_9UNwUZaDsUG
https://images.craigslist.org/00B0B_9UNwUZaDsUG_300x300.jpg
image_information [387, 'Zatchels saddle bag', 8000, 'Setagaya/Shibuya', ' like new', 2, 'im

image_information [397, 'Diesel Thommer Jeans - Brand New', 10000, 'Higashii Azabu Tokyo', ' new', 1, 'images/397_00l0l_ljqmz8kxYfP_1.jpg']
00E0E_3GZ9eG0zvUQ
https://images.craigslist.org/00E0E_3GZ9eG0zvUQ_300x300.jpg
image_information [397, 'Diesel Thommer Jeans - Brand New', 10000, 'Higashii Azabu Tokyo', ' new', 2, 'images/397_00E0E_3GZ9eG0zvUQ_2.jpg']
00g0g_5MjOtd0Zv9j
https://images.craigslist.org/00g0g_5MjOtd0Zv9j_300x300.jpg
image_information [397, 'Diesel Thommer Jeans - Brand New', 10000, 'Higashii Azabu Tokyo', ' new', 3, 'images/397_00g0g_5MjOtd0Zv9j_3.jpg']
00x0x_lpol8ga1pOD
https://images.craigslist.org/00x0x_lpol8ga1pOD_300x300.jpg
image_information [397, 'Diesel Thommer Jeans - Brand New', 10000, 'Higashii Azabu Tokyo', ' new', 4, 'images/397_00x0x_lpol8ga1pOD_4.jpg']
00909_6ZjswPrrHZW
https://images.craigslist.org/00909_6ZjswPrrHZW_300x300.jpg
image_information [397, 'Diesel Thommer Jeans - Brand New', 10000, 'Higashii Azabu Tokyo', ' new', 5, 'images/397_00909_6ZjswPrr

In [18]:
# printing the content of city_data to check we properly
# extracted the information from the posts 
print(city_data)

[[0,
  'Stitch-On Bag for sale! Brand New, Office worker, students OK',
  8000,
  'Ikebukuro',
  ' like new',
  0,
  'images/0_00606_e20sHrQkKN1_0.jpg'],
 [0,
  'Stitch-On Bag for sale! Brand New, Office worker, students OK',
  8000,
  'Ikebukuro',
  ' like new',
  1,
  'images/0_00C0C_jT99a4x5PXI_1.jpg'],
 [0,
  'Stitch-On Bag for sale! Brand New, Office worker, students OK',
  8000,
  'Ikebukuro',
  ' like new',
  2,
  'images/0_00K0K_gsFZSt3KiSJ_2.jpg'],
 [1,
  'Seiko automatic watch - sapphire glass',
  21000,
  'nerima',
  ' new',
  0,
  'images/1_00t0t_a4YR3ouUQcn_0.jpg'],
 [1,
  'Seiko automatic watch - sapphire glass',
  21000,
  'nerima',
  ' new',
  1,
  'images/1_00Y0Y_kQKBjolac1J_1.jpg'],
 [1,
  'Seiko automatic watch - sapphire glass',
  21000,
  'nerima',
  ' new',
  2,
  'images/1_01717_8jdlfNtGIaT_2.jpg'],
 [1,
  'Seiko automatic watch - sapphire glass',
  21000,
  'nerima',
  ' new',
  3,
  'images/1_00N0N_8p4jBJlt8vM_3.jpg'],
 [1,
  'Seiko automatic watch - sapphire g

In [19]:
# creating a dataframe out of the list
city_data = pd.DataFrame(city_data)
# checking the data sits correctly in the new dataframe
city_data.describe(include='all')

,0,1,2,3,4,5,6
count,2017.000000,2017,2017.000000,2017,2017,2017.000000,2017
unique,NaN,383,NaN,135,6,NaN,2017
top,NaN,Top collection (lady),NaN,Tokyo Roppongi,unknown,NaN,images/89_00101_gX9eAUmlW0Z_0.jpg
freq,NaN,69,NaN,254,475,NaN,1
mean,192.125434,NaN,8025.659891,NaN,NaN,3.969757,NaN
std,116.482404,NaN,17873.016363,NaN,NaN,4.602404,NaN
min,0.000000,NaN,0.000000,NaN,NaN,0.000000,NaN
25%,84.000000,NaN,1000.000000,NaN,NaN,1.000000,NaN
50%,186.000000,NaN,3000.000000,NaN,NaN,2.000000,NaN
75%,290.000000,NaN,8000.000000,NaN,NaN,5.000000,NaN


In [20]:
# here we are setting and name for the csv file, assigning proper names to the columns 
# and finally writing the csv file
csv_name = "tokyo_data_10.csv"
city_data.to_csv(csv_name, index= False, header = ["post_id", "post_title", "price", "neighborhood", "condition", "image_id", "uri"])